<a href="https://colab.research.google.com/github/knickka/Comment-Toxicity/blob/main/Comment_toxicity_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/toxci_comment_train.csv'
df = pd.read_csv(path)


In [4]:
df.iloc[43]['comment_text']

'FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!'

In [5]:
df[df.columns[2:]].iloc[43]

toxic            1
severe_toxic     0
obscene          1
threat           0
insult           1
identity_hate    0
Name: 43, dtype: int64

## Preprocess

In [6]:
#imports
from tensorflow.keras.layers import TextVectorization

In [7]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [8]:
MAX_WORDS = 200000 #number of words in our vocab


In [9]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                                  output_sequence_length = 1800,
                                  output_mode = 'int')

In [10]:
vectorizer.adapt(X.values)

In [11]:
vectorized_text = vectorizer(X.values)

In [12]:
len(X)

159571

In [13]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [14]:
#MCSHBP - map, cache, shuffle, batch prefetch from_tensor_slices, from_list

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [15]:
dataset.as_numpy_iterator().next()

(array([[   12,     9,   361, ...,     0,     0,     0],
        [    7, 12924,     4, ...,     0,     0,     0],
        [  345,     7,    59, ...,     0,     0,     0],
        ...,
        [ 4036,  3255,   280, ...,     0,     0,     0],
        [ 1168,    20,     7, ...,     0,     0,     0],
        [12449,  1169,     3, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0]]))

In [16]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [17]:
train_generator = train.as_numpy_iterator()

In [ ]:
#imports